In [ ]:
import urllib
from bs4 import BeautifulSoup as bs
import pandas
import numpy as np
import os
import json
import io
from pprint import pprint
print('imported')

In [ ]:
baseurl = 'http://www.imsdb.com'

theletters = ['0','A','B','C','D','E','F','G','H','I','J','K']
theletters += ['L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']

therawscripts = []

for lett in theletters:
    url = baseurl + '/' +'alphabetical/{0}'.format(lett)
    page = urllib.request.urlopen(url).read()
    soup = bs(page, "html.parser")
    allp = soup.find_all('p')
    for thisp in allp:
        ascript = {}
        thispagelink = thisp.find('a')
        thislink = baseurl+urllib.parse.quote(thispagelink['href'])        
        ascript['scriptpagelink'] = thislink
        ascript['title'] = thisp.text[0:thisp.text.find('(')].strip()
        ascript['version'] = thisp.text[thisp.text.find('(')+1:thisp.text.find(')')].strip()
        ascript['authors'] = thisp.text[thisp.text.find(')')+1:len(thisp.text)].strip()
        ascript['authors'] = ascript['authors'][ascript['authors'].find('Written by')+10:len(ascript['authors'])].strip().split(',')
        therawscripts.append(ascript)
print('Done! Processed {0:,.0f} titles.'.format(len(therawscripts)))

In [ ]:
cnt = 0
notfound = []
thescripts = []
for onescript in therawscripts:
    if (np.mod(cnt, 10) == 0):
        print('Processing title number {0:,.0f}...'.format(cnt))
    url = onescript['scriptpagelink']
    page = urllib.request.urlopen(url).read()
    soup = bs(page, "html.parser")
    skiptitles = ['A.I.']
    if (onescript['title'] in skiptitles):
        print('SKIPPING title {0:,.0f}: {1:}'.format(cnt, onescript['title']))
        notfound.append(onescript)
    else:
        try:
            thatlink = baseurl+soup.find_all('p')[-1].find('a')['href']
            onescript['scriptlink'] = thatlink
            thescripts.append(onescript)
        except TypeError:
            print('Script not found for {0:} and Eeyore is SAD.'.format(onescript['title']))
            notfound.append(onescript)
        except IndexError:
            print('Script not found for {0:}, and no reviews, and Eeyore is SAD.'.format(onescript['title']))
            notfound.append(onescript)
#        print('Title {0:,.0f}: {1:}, script link = {2:}'.format(cnt, onescript['title'], onescript['scriptlink']))
    cnt = cnt + 1
print('Done! Processed {0:,.0f} titles.'.format(len(therawscripts)))
#pprint(notfound)

In [ ]:
with io.open('scripts_notfound.json', 'w', encoding='utf-8') as f:
    json.dump(notfound, f)
print('done')

with io.open('scripts.json', 'w', encoding='utf-8') as f:
    json.dump(thescripts, f)
print('done')

In [ ]:
with io.open('scripts.json', 'r', encoding='utf-8') as f:
    z = json.load(f)
print(len(z))